**NB01-Data-Consoldation**

In [18]:
%load_ext sql
import requests
from bs4 import BeautifulSoup
import pandas as pd
import uuid
import sqlite3

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


**Reading data from csv file from the folder /data/raw**

In [19]:
top_mov_actors_df = pd.read_csv('../data/raw/top_mov_actors.csv', index_col=0)
top_movies_df = pd.read_csv('../data/raw/top_movie.csv',index_col=0)
actors_df = pd.read_csv('../data/raw/actors_list.csv',index_col=0)

**Display the List of Actors with Actor id and their names**

In [20]:
actors_df

,Act_id,Act_name
0,0,Song Kang-ho
1,1,Lee Sun-kyun
2,2,Jo Yeo-jeong
3,3,Choi Woo-sik
4,4,Humphrey Bogart
...,...,...
336,336,Nance O'Neil
337,337,Bessie Love
338,338,Anita Page
339,339,Charles King


**Normalizing the top movie actors names has been removed and actor id has been included, so that the data frame has been normalized and movie key and actor id has been stored**

In [21]:
top_mov_actors_rev_df = pd.merge(top_mov_actors_df, actors_df,
         left_on='actor',
         right_on='Act_name',
         how='left')

In [22]:
# Drop columns multiple column 
top_mov_actors_rev_df = top_mov_actors_rev_df.drop(['actor', 'Act_name'], axis=1)

In [23]:
top_mov_actors_rev_df

,movie_key,Act_id
0,0,0
1,0,1
2,0,2
3,0,3
4,1,4
...,...,...
383,95,336
384,96,337
385,96,338
386,96,339


**Storing the Dataframe consoldated data has been stored into the table by creating a database and tables**

In [24]:
#!pip install pandas numpy lets-plot numerize tqdm sqlalchemy jupysql

import os
import sqlite3
import numpy as np
import pandas as pd

from tqdm.notebook import tqdm, trange
tqdm.pandas()

**Database Creation**

In [25]:
DATA_FOLDER = os.path.join('../data/clean')

# Create a SQLite database in the ../data/ directory if it does not already exist
conn = sqlite3.connect(os.path.join(DATA_FOLDER, './supermovies.db'))

**Table Creation**

In [26]:
conn.execute('''
CREATE TABLE IF NOT EXISTS Movie_Actors
(
    Act_id INTEGER,
    Act_name TEXT
)
''')


conn.execute('''
CREATE TABLE IF NOT EXISTS top_movies (
    rank TEXT,
    title TEXT,
    rel_year INT,
    rating INT,
    movie_key TEXT
)
''')

conn.execute('''
CREATE TABLE IF NOT EXISTS top_mov_actors (
    movie_key TEXT,
    Act_id INTEGER
)
''')


**Data has been loaded into the corresponding tables**

In [27]:
actors_df.to_sql('Movie_Actors', conn, if_exists='append', index=False)
top_movies_df.to_sql('top_movies', conn, if_exists='append', index=False)
top_mov_actors_rev_df.to_sql('top_mov_actors', conn, if_exists='append', index=False)


388

**Checking the data from the database**

In [28]:
%sql sqlite:///../data/supermovies.db --alias supermovies


Connecting to 'supermovies'

In [29]:
%sql supermovies

In [30]:
%%sql

SELECT * FROM movie_actors;

Running query in 'supermovies'

Act_id,Act_name
0,Song Kang-ho
1,Lee Sun-kyun
2,Jo Yeo-jeong
3,Choi Woo-sik
4,Humphrey Bogart
5,Ingrid Bergman
6,Paul Henreid
7,Claude Rains
8,Bette Davis
9,Anne Baxter


In [31]:
%%sql

SELECT * FROM top_mov_actors;


Running query in 'supermovies'

movie_key,Act_id
0,0
0,1
0,2
0,3
1,4
1,5
1,6
1,7
2,8
2,9


In [32]:
%%sql

SELECT * FROM top_movies;

Running query in 'supermovies'

rank,title,rel_year,rating,movie_key
#1,Parasite,2019,99,0
#2,Casablanca,1942,99,1
#3,All About Eve,1950,99,2
#4,On the Waterfront,1954,99,3
#5,Moonlight,2016,98,4
#6,Schindler's List,1993,98,5
#7,It Happened One Night,1934,98,6
#8,Rebecca,1940,98,7
#9,All Quiet on the Western Front,1930,98,8
#10,Sunrise,1927,98,9


In [33]:
%%sql

select rel_year, count(rel_year)
from top_movies
group by rel_year

Running query in 'supermovies'

rel_year,count(rel_year)
1927,6
1929,3
1930,3
1931,3
1932,3
1933,3
1934,3
1935,3
1936,3
1937,3
